In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('Задание 5.0 2 версия.xlsx')

In [3]:
data.rename(columns=
           {
               'FIO': 'name', 'Месяц': 'date', 'Профессия': 'position', 'Разряд': 'classrate', 
               'Оборудование': 'equipment', 'Вредность (балл.)': 'hazard_rate', 
               'Объем производства': 'production_volume', 'Доход': 'revenue'
           }, inplace=True)

In [4]:
data.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638.000000
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638.000000
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638.000000
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396.411765
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790.470588


In [5]:
data['revenue'] = data['revenue'].apply(int)

In [6]:
data.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790


In [7]:
hazard_rate = data.groupby(['classrate', 'hazard_rate'], as_index=False).agg({'revenue': 'min'})

In [8]:
hazard_rate

,classrate,hazard_rate,revenue
0,3,0,12180
1,3,18,15420
2,4,8,15570
3,4,11,16110
4,4,16,17010
5,4,18,17370
6,4,22,18090
7,5,8,17829
8,5,18,19629
9,5,22,20349


In [9]:
hazard_rate[hazard_rate.classrate == 3]

,classrate,hazard_rate,revenue
0,3,0,12180
1,3,18,15420


In [10]:
(15420 - 12180) / (18 - 0)

180.0

In [11]:
hazard_rate[hazard_rate.classrate == 4]

,classrate,hazard_rate,revenue
2,4,8,15570
3,4,11,16110
4,4,16,17010
5,4,18,17370
6,4,22,18090


In [12]:
(17370 - 16110) / (18 - 11)

180.0

In [13]:
hazard_rate[hazard_rate.classrate == 7]

,classrate,hazard_rate,revenue
12,7,0,22059
13,7,18,25299
14,7,22,26019


In [14]:
(25299 - 22059) / (18 - 0)

180.0

**Как мы видим, какие бы баллы вредности мы ни взяли зарплаты из одного и того же (любого) класса, мы получаем удельную разность 180 рублей на 1 балл вредности**

**Теперь, чтобы узнать, зависит ли зарплата всех работников от объема производства, сделаем следующее**

In [15]:
month_revenues = data.groupby('date').agg({'production_volume': 'sum', 'revenue': 'sum'}).sort_values('production_volume')

In [16]:
month_revenues

,production_volume,revenue
date,,
2009-01-01,116000,3977101
2009-10-01,156000,3977101
2009-02-01,158000,3977118
2009-12-01,162000,3977101
2009-03-01,164000,3977101
2009-06-01,190000,4177620
2009-11-01,204000,4318034
2009-09-01,216000,4438259
2009-08-01,236000,4638816


**Как мы видим, базовый план производительности за который платят базовую оплату, лежит между 164000 и 190000**

In [17]:
month_revenues['bonuses'] = (month_revenues.revenue - 3977101).apply(int)

In [18]:
month_revenues

,production_volume,revenue,bonuses
date,,,
2009-01-01,116000,3977101,0
2009-10-01,156000,3977101,0
2009-02-01,158000,3977118,17
2009-12-01,162000,3977101,0
2009-03-01,164000,3977101,0
2009-06-01,190000,4177620,200519
2009-11-01,204000,4318034,340933
2009-09-01,216000,4438259,461158
2009-08-01,236000,4638816,661715


In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
X = month_revenues[['production_volume']].iloc[5:]
y = month_revenues['bonuses'].iloc[5:]

In [21]:
lin_reg = LinearRegression()

In [22]:
lin_reg.fit(X, y)

LinearRegression()

In [23]:
round(lin_reg.coef_[0], 3)

10.026

In [24]:
-int(lin_reg.intercept_)

1704391

**План производительности, после которого будут доплаты, равен 1704391**


**За каждый пункт перевыполнения прлана производительности работодатель выделяет 10.026 рублей. Нужно посчитать, как эти премии распределены между работниками, но это сделаем позже**

**Теперь посчитаем зависимость оклада от разряда**

**Для этого возьмем месяцы, в которых получались только базовые оплаты**

In [25]:
data

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790
...,...,...,...,...,...,...,...,...
2395,Яковлев Д. И.,2009-08-01,Подручный вальцовщика,4,Прошивной стан,18,1180,21800
2396,Яковлев Д. И.,2009-09-01,Подручный вальцовщика,4,Прошивной стан,18,1080,20885
2397,Яковлев Д. И.,2009-10-01,Подручный вальцовщика,4,Прошивной стан,18,780,18782
2398,Яковлев Д. И.,2009-11-01,Подручный вальцовщика,4,Прошивной стан,18,1020,20337


In [26]:
classrate = data[data.date == '2009-01-01'].groupby(['hazard_rate', 'classrate'], as_index=False).agg({'revenue': 'min'})

In [27]:
classrate

,hazard_rate,classrate,revenue
0,0,3,12180
1,0,7,22059
2,8,4,15570
3,8,5,17829
4,11,4,16110
5,16,4,17010
6,18,3,15420
7,18,4,17370
8,18,5,19629
9,18,6,22260


In [28]:
classrate['zero_hazardness_revenue'] = classrate.revenue - classrate.hazard_rate * 180

In [29]:
classrate.set_index('hazard_rate')

,classrate,revenue,zero_hazardness_revenue
hazard_rate,,,
0,3,12180,12180
0,7,22059,22059
8,4,15570,14130
8,5,17829,16389
11,4,16110,14130
16,4,17010,14130
18,3,15420,12180
18,4,17370,14130
18,5,19629,16389


In [30]:
(22059 / 12180) ** (1 / (7 - 3))

1.160071167929978

In [31]:
(16389 / 14130) ** (1 / (5 - 4))

1.1598726114649682

In [32]:
(14130 / 12180) ** (1 / (4 - 3))

1.1600985221674878

In [33]:
(22059 / 14130) ** (1 / (7 - 4))

1.1600620499941403

**Очевидно, что увеличение на 1 разряд увеличивает ТЕКУШЕЕ значение оклада в 1.16**

**Посчитаем базовый оклад для 0 разрада и 0 вредности**

In [34]:
classrate.head()

,hazard_rate,classrate,revenue,zero_hazardness_revenue
0,0,3,12180,12180
1,0,7,22059,22059
2,8,4,15570,14130
3,8,5,17829,16389
4,11,4,16110,14130


In [35]:
classrate['base'] = (classrate.zero_hazardness_revenue / (1.16) ** classrate.classrate).apply(int)

In [36]:
classrate.head()

,hazard_rate,classrate,revenue,zero_hazardness_revenue,base
0,0,3,12180,12180,7803
1,0,7,22059,22059,7805
2,8,4,15570,14130,7803
3,8,5,17829,16389,7803
4,11,4,16110,14130,7803


In [37]:
classrate.base.mean()

7803.8

**Самый базовый оклад равен 7804 рубля**

**Сделаем теперь таблицу, где указаны все подсчитанные зарплаты по окладу, разряду и опасности**

In [38]:
indviduals = data

In [39]:
indviduals['counted_1'] = (7803.8 * 1.16 ** indviduals.classrate + 180 * indviduals.hazard_rate).apply(round)

In [40]:
indviduals['difference_1'] = indviduals.revenue - indviduals.counted_1

In [41]:
indviduals

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,counted_1,difference_1
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638,15421,1217
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638,15421,1217
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638,15421,1217
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396,15421,3975
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790,15421,4369
...,...,...,...,...,...,...,...,...,...,...
2395,Яковлев Д. И.,2009-08-01,Подручный вальцовщика,4,Прошивной стан,18,1180,21800,17370,4430
2396,Яковлев Д. И.,2009-09-01,Подручный вальцовщика,4,Прошивной стан,18,1080,20885,17370,3515
2397,Яковлев Д. И.,2009-10-01,Подручный вальцовщика,4,Прошивной стан,18,780,18782,17370,1412
2398,Яковлев Д. И.,2009-11-01,Подручный вальцовщика,4,Прошивной стан,18,1020,20337,17370,2967


**Сначала посчиаем индивидуальные надбавки, для этого нужно взять месяцы без премий**

In [42]:
personals = indviduals[indviduals.date == '2009-01-01'] \
.loc[:, ['name', 'difference_1', 'classrate', 'hazard_rate']]

In [43]:
personals.head()

,name,difference_1,classrate,hazard_rate
0,Абрамов Е. Д.,1217,3,18
12,Абрамов Ф. М.,607,3,18
24,Агафонов С. И.,7,6,18
36,Аксенов Е. Н.,4,7,22
48,Александров А. М.,-1,3,18


**Заменим индексы и заменим значения которые получились в пределах погрешности 10 на 0**

In [44]:
personals = personals.reset_index().drop(columns='index')

In [45]:
personals.head()

,name,difference_1,classrate,hazard_rate
0,Абрамов Е. Д.,1217,3,18
1,Абрамов Ф. М.,607,3,18
2,Агафонов С. И.,7,6,18
3,Аксенов Е. Н.,4,7,22
4,Александров А. М.,-1,3,18


In [46]:
for i in personals.index:
    if personals.difference_1.iloc[i] < 10:
        personals.difference_1.iloc[i] = 0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [47]:
personals.head()

,name,difference_1,classrate,hazard_rate
0,Абрамов Е. Д.,1217,3,18
1,Абрамов Ф. М.,607,3,18
2,Агафонов С. И.,0,6,18
3,Аксенов Е. Н.,0,7,22
4,Александров А. М.,0,3,18


**Посмотрим, как себя ведут эти разности**

In [48]:
personals.sort_values('difference_1', ascending=False).head()

,name,difference_1,classrate,hazard_rate
152,Сергеев Р. Е.,2211,7,18
40,Гришин А. А.,2211,7,18
194,Шаповалов А. Р.,2211,7,22
188,Чернов И. А.,2211,7,22
181,Филиппов И. П.,2211,7,0


In [49]:
personals.groupby(['hazard_rate', 'classrate', 'difference_1']).count()

name
hazard_rate classrate difference_1      
0           3         0                6
                      607              2
                      1217             1
            7         0                4
                      1107             4
                      2211             1
8           4         0               10
                      706              3
            5         0                4
                      818              5
                      1638             5
11          4         0                3
                      706              1
                      1412             1
16          4         0                5
                      706              2
18          3         0               15
                      607              6
                      1217             5
            4         0                5
                      706              9
                      1412             2
            5         0                5
                      818              5
            6         0                9
                      958              8
                      1909             5
            7         0                7
                      1107             9
                      2211             3
22          4         0                8
                      706              4
                      1412             3
            5         0               10
                      818              4
                      1638             3
            6         0                4
                      958              4
            7         0                5
                      1107             3
                      2211             2

**Как мы видим, закономерности по разрядам и баллам вредности есть, но есть и другие данные, которые мы не обладаем, поэтому пропишем индивидуальные надбавки для каждого работника**

In [50]:
personals = personals.groupby(['name', 'hazard_rate', 'classrate', 'difference_1'], as_index=False).min()
personals

,name,hazard_rate,classrate,difference_1
0,Абрамов Е. Д.,18,3,1217
1,Абрамов Ф. М.,18,3,607
2,Агафонов С. И.,18,6,0
3,Аксенов Е. Н.,22,7,0
4,Александров А. М.,18,3,0
...,...,...,...,...
195,Шестаков Е. А.,18,3,0
196,Щербаков А. Е.,8,4,706
197,Щербаков А. М.,18,5,0
198,Щукин К. С.,8,5,818


In [51]:
indviduals = indviduals.drop(columns=['difference_1']).merge(personals.loc[:, ['name', 'difference_1']], on='name')
indviduals

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,counted_1,difference_1
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638,15421,1217
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638,15421,1217
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638,15421,1217
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396,15421,1217
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790,15421,1217
...,...,...,...,...,...,...,...,...,...,...
2395,Яковлев Д. И.,2009-08-01,Подручный вальцовщика,4,Прошивной стан,18,1180,21800,17370,1412
2396,Яковлев Д. И.,2009-09-01,Подручный вальцовщика,4,Прошивной стан,18,1080,20885,17370,1412
2397,Яковлев Д. И.,2009-10-01,Подручный вальцовщика,4,Прошивной стан,18,780,18782,17370,1412
2398,Яковлев Д. И.,2009-11-01,Подручный вальцовщика,4,Прошивной стан,18,1020,20337,17370,1412


In [52]:
indviduals['counted_2'] = indviduals.counted_1 + indviduals.difference_1
indviduals['difference_2'] = indviduals.revenue - indviduals.counted_2
indviduals.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,counted_1,difference_1,counted_2,difference_2
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638,15421,1217,16638,0
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638,15421,1217,16638,0
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638,15421,1217,16638,0
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396,15421,1217,16638,2758
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790,15421,1217,16638,3152


In [53]:
for i in indviduals.index:
    if indviduals.difference_2.iloc[i] < 10:
        indviduals.difference_2.iloc[i] = 0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [54]:
indviduals.drop(columns=['counted_1', 'counted_2'], inplace=True)

In [55]:
indviduals.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638,1217,0
1,Абрамов Е. Д.,2009-02-01,Уборщик,3,Пилигримовые станы,18,790,16638,1217,0
2,Абрамов Е. Д.,2009-03-01,Уборщик,3,Пилигримовые станы,18,820,16638,1217,0
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396,1217,2758
4,Абрамов Е. Д.,2009-05-01,Уборщик,3,Пилигримовые станы,18,1250,19790,1217,3152


In [56]:
april = indviduals[indviduals.date == '2009-04-01']
april.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396,1217,2758
15,Абрамов Ф. М.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,18662,607,2634
27,Агафонов С. И.,2009-04-01,Вальцовщик,6,Пилигримовые станы,18,1200,26175,0,3922
39,Аксенов Е. Н.,2009-04-01,Нагревальщик металла,7,Нагревательные печи,22,1200,30561,0,4546
51,Александров А. М.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,17927,0,2506


In [57]:
month_revenues.sort_values('date')

,production_volume,revenue,bonuses
date,,,
2009-01-01,116000,3977101,0
2009-02-01,158000,3977118,17
2009-03-01,164000,3977101,0
2009-04-01,240000,4678921,701820
2009-05-01,250000,4779183,802082
2009-06-01,190000,4177620,200519
2009-07-01,280000,5079962,1102861
2009-08-01,236000,4638816,661715
2009-09-01,216000,4438259,461158


In [58]:
april['total_bonuses'] = 701820

C:\Users\6A02~1\AppData\Local\Temp/ipykernel_11676/2970850008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  april['total_bonuses'] = 701820


In [59]:
april.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2,total_bonuses
3,Абрамов Е. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,19396,1217,2758,701820
15,Абрамов Ф. М.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,18662,607,2634,701820
27,Агафонов С. И.,2009-04-01,Вальцовщик,6,Пилигримовые станы,18,1200,26175,0,3922,701820
39,Аксенов Е. Н.,2009-04-01,Нагревальщик металла,7,Нагревательные печи,22,1200,30561,0,4546,701820
51,Александров А. М.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,17927,0,2506,701820


In [60]:
april['interest'] = 100 * april.difference_2 / 701820

C:\Users\6A02~1\AppData\Local\Temp/ipykernel_11676/251202126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  april['interest'] = 100 * april.difference_2 / 701820


In [61]:
april.sort_values('interest').head(10)

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2,total_bonuses,interest
1563,Осипов М. В.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,17927,0,2506,701820,0.357072
1371,Наумов О. В.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687,0,2506,701820,0.357072
1347,Муравьев В. Б.,2009-04-01,Подручный вальцовщика,3,Пилигримовые станы,18,1200,17927,0,2506,701820,0.357072
1683,Потапов М. И.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,17927,0,2506,701820,0.357072
471,Григорьев А. Т.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687,0,2506,701820,0.357072
1719,Румянцев И. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,17927,0,2506,701820,0.357072
1755,Самсонов К. Д.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,17927,0,2506,701820,0.357072
1215,Львов А. С.,2009-04-01,Уборщик,3,Вспомогательные помещения,0,1200,14687,0,2506,701820,0.357072
219,Быков Л. Н.,2009-04-01,Подручный вальцовщика,3,Пилигримовые станы,18,1200,17927,0,2506,701820,0.357072
987,Краснов И. М.,2009-04-01,Уборщик,3,Пилигримовые станы,18,1200,17927,0,2506,701820,0.357072


In [62]:
april.groupby(['classrate', 'hazard_rate', 'interest']).agg({'revenue': 'count'})

revenue
classrate hazard_rate interest         
3         0           0.357072        6
                      0.375310        2
                      0.392978        1
          18          0.357072       15
                      0.375310        6
                      0.392978        5
4         8           0.414494       10
                      0.435297        3
          11          0.414494        3
                      0.435297        1
                      0.456100        1
          16          0.414494        5
                      0.435297        2
          18          0.414494        5
                      0.435297        9
                      0.456100        2
          22          0.414494        8
                      0.435297        4
                      0.456100        3
5         8           0.480608        4
                      0.504830        5
                      0.528768        5
          18          0.480608        5
                      0.504830        5
          22          0.480608       10
                      0.504830        4
                      0.528768        3
6         18          0.558833        9
                      0.585763        8
                      0.613690        5
          22          0.558833        4
                      0.585763        4
7         0           0.647744        4
                      0.679519        4
                      0.711721        1
          18          0.647744        7
                      0.679519        9
                      0.711721        3
          22          0.647744        5
                      0.679519        3
                      0.711721        2

**Убедимся, что процент от общих бонусов для каждого работника персонален**

**Проверим еще на одном месяце**

In [63]:
july = indviduals[indviduals.date == '2009-07 -01']
july.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2
6,Абрамов Е. Д.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,20972,1217,4334
18,Абрамов Ф. М.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,20166,607,4138
30,Агафонов С. И.,2009-07-01,Вальцовщик,6,Пилигримовые станы,18,1400,28413,0,6160
42,Аксенов Е. Н.,2009-07-01,Нагревальщик металла,7,Нагревательные печи,22,1400,33157,0,7142
54,Александров А. М.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,19360,0,3939


In [64]:
month_revenues.sort_values('date')

,production_volume,revenue,bonuses
date,,,
2009-01-01,116000,3977101,0
2009-02-01,158000,3977118,17
2009-03-01,164000,3977101,0
2009-04-01,240000,4678921,701820
2009-05-01,250000,4779183,802082
2009-06-01,190000,4177620,200519
2009-07-01,280000,5079962,1102861
2009-08-01,236000,4638816,661715
2009-09-01,216000,4438259,461158


In [65]:
july['total_bonuses'] = 1102861

C:\Users\6A02~1\AppData\Local\Temp/ipykernel_11676/273828348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july['total_bonuses'] = 1102861


In [66]:
july.head()

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2,total_bonuses
6,Абрамов Е. Д.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,20972,1217,4334,1102861
18,Абрамов Ф. М.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,20166,607,4138,1102861
30,Агафонов С. И.,2009-07-01,Вальцовщик,6,Пилигримовые станы,18,1400,28413,0,6160,1102861
42,Аксенов Е. Н.,2009-07-01,Нагревальщик металла,7,Нагревательные печи,22,1400,33157,0,7142,1102861
54,Александров А. М.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,19360,0,3939,1102861


In [67]:
july['interest'] = 100 * july.difference_2 / 1102861

C:\Users\6A02~1\AppData\Local\Temp/ipykernel_11676/1416909218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july['interest'] = 100 * july.difference_2 / 1102861


In [68]:
july.sort_values('interest').head(10)

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2,total_bonuses,interest
1566,Осипов М. В.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,19360,0,3939,1102861,0.357162
1374,Наумов О. В.,2009-07-01,Уборщик,3,Вспомогательные помещения,0,1400,16120,0,3939,1102861,0.357162
1350,Муравьев В. Б.,2009-07-01,Подручный вальцовщика,3,Пилигримовые станы,18,1400,19360,0,3939,1102861,0.357162
1686,Потапов М. И.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,19360,0,3939,1102861,0.357162
474,Григорьев А. Т.,2009-07-01,Уборщик,3,Вспомогательные помещения,0,1400,16120,0,3939,1102861,0.357162
1722,Румянцев И. Д.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,19360,0,3939,1102861,0.357162
1758,Самсонов К. Д.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,19360,0,3939,1102861,0.357162
1218,Львов А. С.,2009-07-01,Уборщик,3,Вспомогательные помещения,0,1400,16120,0,3939,1102861,0.357162
222,Быков Л. Н.,2009-07-01,Подручный вальцовщика,3,Пилигримовые станы,18,1400,19360,0,3939,1102861,0.357162
990,Краснов И. М.,2009-07-01,Уборщик,3,Пилигримовые станы,18,1400,19360,0,3939,1102861,0.357162


In [69]:
july_interests = july.groupby(['name'], as_index=False).agg({'interest': 'mean'})
july_interests

,name,interest
0,Абрамов Е. Д.,0.392978
1,Абрамов Ф. М.,0.375206
2,Агафонов С. И.,0.558547
3,Аксенов Е. Н.,0.647588
4,Александров А. М.,0.357162
...,...,...
195,Шестаков Е. А.,0.357162
196,Щербаков А. Е.,0.435232
197,Щербаков А. М.,0.480659
198,Щукин К. С.,0.504869


In [70]:
april_interests = april.groupby(['name'], as_index=False).agg({'interest': 'mean'})
april_interests

,name,interest
0,Абрамов Е. Д.,0.392978
1,Абрамов Ф. М.,0.375310
2,Агафонов С. И.,0.558833
3,Аксенов Е. Н.,0.647744
4,Александров А. М.,0.357072
...,...,...
195,Шестаков Е. А.,0.357072
196,Щербаков А. Е.,0.435297
197,Щербаков А. М.,0.480608
198,Щукин К. С.,0.504830


**Как и в случае с надбавками, некая закономерность в комисии от объема переработок есть, но информации недостаточно, поэтому надо поименно зафиксировать комиссию от переработок**

In [71]:
july_interests.head()

,name,interest
0,Абрамов Е. Д.,0.392978
1,Абрамов Ф. М.,0.375206
2,Агафонов С. И.,0.558547
3,Аксенов Е. Н.,0.647588
4,Александров А. М.,0.357162


In [72]:
april_interests.head()

,name,interest
0,Абрамов Е. Д.,0.392978
1,Абрамов Ф. М.,0.375310
2,Агафонов С. И.,0.558833
3,Аксенов Е. Н.,0.647744
4,Александров А. М.,0.357072


In [73]:
july_interests['interest'] = july_interests.interest.apply(lambda x: round(x, 3))
april_interests['interest'] = april_interests.interest.apply(lambda x: round(x, 3))

In [74]:
compare = july_interests.merge(april_interests, on='name')

In [75]:
compare['difference'] = (compare.interest_x - compare.interest_y).apply(int)

In [76]:
compare

,name,interest_x,interest_y,difference
0,Абрамов Е. Д.,0.393,0.393,0
1,Абрамов Ф. М.,0.375,0.375,0
2,Агафонов С. И.,0.559,0.559,0
3,Аксенов Е. Н.,0.648,0.648,0
4,Александров А. М.,0.357,0.357,0
...,...,...,...,...
195,Шестаков Е. А.,0.357,0.357,0
196,Щербаков А. Е.,0.435,0.435,0
197,Щербаков А. М.,0.481,0.481,0
198,Щукин К. С.,0.505,0.505,0


In [77]:
interests = compare.loc[:, ['name', 'interest_x']].rename(columns={'interest_x': 'interest'})

**Таблица с поименными долями от общей суммы бонусов**

In [78]:
interests

,name,interest
0,Абрамов Е. Д.,0.393
1,Абрамов Ф. М.,0.375
2,Агафонов С. И.,0.559
3,Аксенов Е. Н.,0.648
4,Александров А. М.,0.357
...,...,...
195,Шестаков Е. А.,0.357
196,Щербаков А. Е.,0.435
197,Щербаков А. М.,0.481
198,Щукин К. С.,0.505


**Доли от общей суммы бонусов нужно перевести в коэффициент умножения на личную переработку**
**Считается это легко: умножаем долю на  2 * 10,026**

In [79]:
interests['interest'] = interests['interest'].apply(lambda x: x * 2 * 10.026)

In [80]:
interests

,name,interest
0,Абрамов Е. Д.,7.880436
1,Абрамов Ф. М.,7.519500
2,Агафонов С. И.,11.209068
3,Аксенов Е. Н.,12.993696
4,Александров А. М.,7.158564
...,...,...
195,Шестаков Е. А.,7.158564
196,Щербаков А. Е.,8.722620
197,Щербаков А. М.,9.645012
198,Щукин К. С.,10.126260


In [81]:
personals = personals.merge(interests, on='name')

In [82]:
personals.head()

,name,hazard_rate,classrate,difference_1,interest
0,Абрамов Е. Д.,18,3,1217,7.880436
1,Абрамов Ф. М.,18,3,607,7.519500
2,Агафонов С. И.,18,6,0,11.209068
3,Аксенов Е. Н.,22,7,0,12.993696
4,Александров А. М.,18,3,0,7.158564


In [83]:
personals = personals.rename(columns={'difference_1': 'allowance'})

In [84]:
personals.head()

,name,hazard_rate,classrate,allowance,interest
0,Абрамов Е. Д.,18,3,1217,7.880436
1,Абрамов Ф. М.,18,3,607,7.519500
2,Агафонов С. И.,18,6,0,11.209068
3,Аксенов Е. Н.,22,7,0,12.993696
4,Александров А. М.,18,3,0,7.158564


In [85]:
personals.groupby(['interest', 'allowance', 'classrate', 'hazard_rate']).count()

name
interest  allowance classrate hazard_rate      
7.158564  0         3         0               6
                              18             15
7.519500  607       3         0               2
                              18              6
7.880436  1217      3         0               1
                              18              5
8.301528  0         4         8              10
                              11              3
                              16              5
                              18              5
                              22              8
8.722620  706       4         8               3
                              11              1
                              16              2
                              18              9
                              22              4
9.143712  1412      4         11              1
                              18              2
                              22              3
9.645012  0         5         8               4
                              18              5
                              22             10
10.126260 818       5         8               5
                              18              5
                              22              4
10.607508 1638      5         8               5
                              22              3
11.209068 0         6         18              9
                              22              4
11.750472 958       6         18              8
                              22              4
12.311928 1909      6         18              5
12.993696 0         7         0               4
                              18              7
                              22              5
13.635360 1107      7         0               4
                              18              9
                              22              3
14.277024 2211      7         0               1
                              18              3
                              22              2

**Теперь проверим нашу формулу на адекватность**

In [86]:
personals.head(1)

,name,hazard_rate,classrate,allowance,interest
0,Абрамов Е. Д.,18,3,1217,7.880436


In [87]:
indviduals.head(1)

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,difference_1,difference_2
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638,1217,0


In [88]:
indviduals = indviduals.drop(columns=['difference_1', 'difference_2']) \
.merge(personals.loc[:, ['name', 'allowance', 'interest']])

In [89]:
indviduals.head(1)

,name,date,position,classrate,equipment,hazard_rate,production_volume,revenue,allowance,interest
0,Абрамов Е. Д.,2009-01-01,Уборщик,3,Пилигримовые станы,18,580,16638,1217,7.880436


In [96]:
indviduals['counted_revenue'] = (7803.8 * 1.16 ** indviduals.classrate + 180 * indviduals.hazard_rate + \
indviduals.allowance + (indviduals.production_volume - 850) *  \
(indviduals.production_volume > 850).apply(int) * indviduals.interest).apply(round)

In [98]:
indviduals['compare'] = (indviduals.revenue - indviduals.counted_revenue).apply(round)

In [101]:
indviduals['compare'].unique()

array([ 0, -1,  1,  2,  7, -2,  4, -5,  3, -3,  5, -4, -6], dtype=int64)

**Очевидно, что разность реального дохода и предсказанного в пределах погрешности 10, а значит, формула работает правильно**

**Напоследок, хочу сказать, что очень хотел установить связь между индивидуальными надбавками и коэффициентом за переработку, скажем так. Но возможно, данных недостаточно**